In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [2]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for buying a space with 1+ beds and 2 baths
url = "https://www.royallepage.ca/en/search/homes/on/toronto/?search_str=Toronto%2C+ON%2C+CAN&csrfmiddlewaretoken=nQW9yRIMkogQ8o9McuiC1lKwNQa2HAP0OiQSexsBsjeNDISnDvIE3M4lVoZhRHuZ&property_type=&house_type=&features=&listing_type=&lat=43.65352400000006&lng=-79.38390699999997&upper_lat=&upper_lng=&lower_lat=&lower_lng=&bypass=&radius=&zoom=&display_type=gallery-view&travel_time=&travel_time_min=30&travel_time_mode=drive&travel_time_congestion=&da_id=&segment_id=&tier2=False&tier2_proximity=0&address=Toronto&method=homes&address_type=city&city_name=Toronto&prov_code=ON&school_id=&min_price=0&max_price=2000000&min_leaseprice=0&max_leaseprice=5000%2B&beds=1%2B&baths=2&transactionType=SALE&keyword=&sortby="

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [3]:
# Go through all pages to collect data
house_listings = []
for x in range(1, 23):
    html = browser.html
    soup = bs(html, 'html.parser')
    links = soup.find_all(class_='address-1')
    
    # Get all address listings for buying    
    print('Gathering data from page:', x)
    for link in links:
        house_listings.append(link.a['href'])
    browser.find_by_css('.next').click()

# Number of listings
len(house_listings)


Gathering data from page: 1
Gathering data from page: 2
Gathering data from page: 3
Gathering data from page: 4
Gathering data from page: 5
Gathering data from page: 6
Gathering data from page: 7
Gathering data from page: 8
Gathering data from page: 9
Gathering data from page: 10
Gathering data from page: 11
Gathering data from page: 12
Gathering data from page: 13
Gathering data from page: 14
Gathering data from page: 15
Gathering data from page: 16
Gathering data from page: 17
Gathering data from page: 18
Gathering data from page: 19
Gathering data from page: 20
Gathering data from page: 21
Gathering data from page: 22


1012

In [4]:
browser.quit()

In [5]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)


# Visit each listing and grab relevant data
house_listing_info = []
for i in range(len(house_listings)):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 

    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    
    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text

    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    values = building_feature_section.find_all('span', class_="value")
    
    # Style of house
    style_type = values[0].text.strip(":")

    houses['address'] = address
    houses['price'] = price
    houses['number_of_beds'] = bed_count
    houses['number_of_baths'] = bath_count
    houses['style'] = style_type
    
    
    house_listing_info.append(houses)
    
browser.quit()


AttributeError: 'NoneType' object has no attribute 'find'

In [6]:
house_listing_info

[{'address': '#4002 -1 YORKVILLE AVE, Toronto, Ontario, M4W 1L1',
  'price': '$1,169,000',
  'number_of_beds': '2 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Apartment'},
 {'address': '#4204 -12 YORK ST, Toronto, Ontario, M5J 0A9',
  'price': '$999,000',
  'number_of_beds': '3+1 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Apartment'},
 {'address': '7 PENDEEN AVE, Toronto, Ontario, M6N 2P4',
  'price': '$899,000',
  'number_of_beds': '3+1 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Detached'},
 {'address': '431 GLENHOLME AVE, Toronto, Ontario, M6E 3E9',
  'price': '$849,000',
  'number_of_beds': '3+2 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Detached'},
 {'address': '#611 -15 SINGER CRT, Toronto, Ontario, M2K 0B1',
  'price': '$665,000',
  'number_of_beds': '1+1 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Apartment'},
 {'address': '#1010 -6 EVA RD, Toronto, Ontario, M9C 0B1',
  'price': '$634,900',
  'number_of_beds': '2 Beds',
  'number_of_baths': '

In [7]:
import csv

try:
    with open('./Resources/housing.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['address', 'price', 'number_of_beds', 'number_of_baths', 'style'])
        writer.writeheader()
        for data in house_listing_info:
            writer.writerow(data)
except IOError:
    print("I/O error")